In [2]:
import requests
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pytz
import pandas as pd
import matplotlib.dates as mdates
import argparse

In [3]:
price_area = 'dk1'.upper()

url = "https://api.energidataservice.dk/dataset/Elspotprices"
params = {
            'start': datetime.now(pytz.timezone('Europe/Copenhagen')).strftime('%Y-%m-%d'),
            'end': (datetime.now(pytz.timezone('Europe/Copenhagen')) + timedelta(days=1)).strftime('%Y-%m-%d'),
            'filter': f'{{"PriceArea":"{price_area}"}}',
            'sort': 'HourDK'
        }

response = requests.get(url, params=params)

## Og så tester vi koden

In [7]:
data = response.json().get('records', [])

#if not data:
#    raise ValueError(f"Ingen prisdata modtaget fra API'et for region {region}")
                
# Konvertér til pandas DataFrame for nemmere databehandling
df = pd.DataFrame(data)
df['HourDK'] = pd.to_datetime(df['HourDK'])

# Konvertér øre/kWh til kr/kWh (SpotPriceDKK er i kr/MWh)
df['SpotPriceDKK_kWh'] = df['SpotPriceDKK'] / 1000

# Tilføj afgifter (dette er eksempelværdier - brug aktuelle satser)
moms_rate = 0.25  # 25% moms
elafgift = 0.699  # kr/kWh (eksempelværdi)
systemtarif = 0.054  # kr/kWh (eksempelværdi)
nettarif = 0.213  # kr/kWh (eksempelværdi)

# Beregn total pris inklusive afgifter
df['TotalPris'] = df['SpotPriceDKK_kWh'] + elafgift + systemtarif + nettarif
df['TotalPrisMedMoms'] = df['TotalPris'] * (1 + moms_rate)

In [8]:
df

,HourUTC,HourDK,PriceArea,SpotPriceDKK,SpotPriceEUR,SpotPriceDKK_kWh,TotalPris,TotalPrisMedMoms
0,2025-02-24T23:00:00,2025-02-25 00:00:00,DK1,645.049988,86.459999,0.64505,1.61105,2.013812
1,2025-02-25T00:00:00,2025-02-25 01:00:00,DK1,670.789978,89.910004,0.67079,1.63679,2.045987
2,2025-02-25T01:00:00,2025-02-25 02:00:00,DK1,689.510010,92.419998,0.68951,1.65551,2.069388
3,2025-02-25T02:00:00,2025-02-25 03:00:00,DK1,689.659973,92.440002,0.68966,1.65566,2.069575
4,2025-02-25T03:00:00,2025-02-25 04:00:00,DK1,703.690002,94.320000,0.70369,1.66969,2.087113
5,2025-02-25T04:00:00,2025-02-25 05:00:00,DK1,772.179993,103.500000,0.77218,1.73818,2.172725
6,2025-02-25T05:00:00,2025-02-25 06:00:00,DK1,1044.489990,140.000000,1.04449,2.01049,2.513112
7,2025-02-25T06:00:00,2025-02-25 07:00:00,DK1,1210.790039,162.289993,1.21079,2.17679,2.720988
8,2025-02-25T07:00:00,2025-02-25 08:00:00,DK1,1261.219971,169.050003,1.26122,2.22722,2.784025
9,2025-02-25T08:00:00,2025-02-25 09:00:00,DK1,1127.750000,151.160004,1.12775,2.09375,2.617188
